# How to run reV using HSDS
## Running reV Gen+Econ

[reV Gen](https://nrel.github.io/reV/reV/reV.generation.generation.html#reV.generation.generation.Gen) and [reV Econ](https://nrel.github.io/reV/reV/reV.econ.econ.html#reV.econ.econ.Econ) can be run locally using the [HDF Groups](https://www.hdfgroup.org) Highly Scalable Distributed Service (HSDS) to access resource .h5 file stored in the cloud (currenly on Amazon Web Services -- AWS).

reV use [PySAM](https://pysam.readthedocs.io/en/latest/) to
compute technologically specific capcity factor means and profiles (`Gen`) and mean levelized cost of energy (`Econ`). `reV`
uses ``SAM`` technology terms and input configuration files.


NOTE: In all of these examples, the ``sam_file`` input points to files in
the reV test directory [`TESTDATADIR`](https://github.com/NREL/reV/tree/master/tests/data) that may not be copied in your install. You may want to download the relevant SAM system configs from that directory and point the ``sam_file`` variable to the correct filepath on your computer.

Clone the github - https://github.com/NREL/reV.git and point the TESTDATADIR to tests/data location in your computer.

### Using HSDS

The Highly Scalable Distributed Service (HSDS) is a cloud optimized API to enable access to .h5 files hosted on [AWS](https://registry.opendata.aws). The HSDS software was developed by the [HDF Group](https://www.hdfgroup.org/) and is hosted on Amazon Web Services (AWS) using a combination of EC2 (Elastic Compute) and S3 (Scalable Storage Service). You can read more about the HSDS service [in this slide deck](https://www.slideshare.net/HDFEOS/hdf-cloud-services).


#### Setting up HSDS

To get started install the h5pyd library:

```bash
pip install h5pyd
```

Next, configure h5pyd by running ``hsconfigure`` from the command line, or by
creating a configuration file at ``~/.hscfg``:

```bash
hsconfigure

hs_endpoint = https://developer.nrel.gov/api/hsds
hs_username =
hs_password =
hs_api_key = 3K3JQbjZmWctY0xmIfSYvYgtIcM3CN0cb1Y2w9bf
```

**The example API key here is for demonstration and is rate-limited per IP. To
get your own API key, visit https://developer.nrel.gov/signup/**

**Please note that our HSDS service is for demonstration purposes only, if you
would like to use HSDS for production runs of reV please setup your own
service: https://github.com/HDFGroup/hsds and point it to our public HSDS
bucket: s3://nrel-pds-hsds**

#### Using HSDS with reV

Once h5pyd has been installed and configured, [rex](https://github.com/nrel/rex) can pull data directly from AWS using [HSDS](https://github.com/NREL/hsds-examples)

To access the resource data used by reV (NSRDB or WTK) you have to turn on the
``hsds`` flag in the `rex`'s [resource handlers](https://nrel.github.io/rex/rex/rex.renewable_resource.html):

In [1]:
import rex
from rex import init_logger

init_logger('reV', log_level='INFO')

nsrdb_file = '/nrel/nsrdb/v3/nsrdb_2013.h5'
with rex.Resource(nsrdb_file, hsds=True) as f:
    meta = f.meta
    time_index = f.time_index

display(meta.head())
display(time_index)

,latitude,longitude,elevation,timezone,country,state,county,urban,population,landcover
gid,,,,,,,,,,
0,-19.99,-175.259995,0.0,13,None,None,None,None,-9999,210
1,-19.99,-175.220001,0.0,13,None,None,None,None,-9999,210
2,-19.99,-175.179993,0.0,13,None,None,None,None,-9999,210
3,-19.99,-175.139999,0.0,13,None,None,None,None,-9999,210
4,-19.99,-175.100006,0.0,13,None,None,None,None,-9999,210


DatetimeIndex(['2013-01-01 00:00:00+00:00', '2013-01-01 00:30:00+00:00',
               '2013-01-01 01:00:00+00:00', '2013-01-01 01:30:00+00:00',
               '2013-01-01 02:00:00+00:00', '2013-01-01 02:30:00+00:00',
               '2013-01-01 03:00:00+00:00', '2013-01-01 03:30:00+00:00',
               '2013-01-01 04:00:00+00:00', '2013-01-01 04:30:00+00:00',
               ...
               '2013-12-31 19:00:00+00:00', '2013-12-31 19:30:00+00:00',
               '2013-12-31 20:00:00+00:00', '2013-12-31 20:30:00+00:00',
               '2013-12-31 21:00:00+00:00', '2013-12-31 21:30:00+00:00',
               '2013-12-31 22:00:00+00:00', '2013-12-31 22:30:00+00:00',
               '2013-12-31 23:00:00+00:00', '2013-12-31 23:30:00+00:00'],
              dtype='datetime64[ns, UTC]', length=17520, freq=None)

#### windpower

Compute wind capacity factors for a given set of latitude and longitude
coordinates:

In [6]:
TESTDATADIR='/Users/lijjumathew/Code/SMU/reV/tests/data'

In [3]:
import os
import numpy as np
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

lat_lons = np.array([[ 41.75, -73.66],
                     [ 40.05, -73.74],
                     [ 40.97, -73.78],
                     [ 41.65, -73.74],
                     [ 40.25, -73.7 ],
                     [ 41.05, -73.78]])

res_file = '/nrel/wtk/conus/wtk_conus_2012.h5'  # HSDS 'file' path
sam_file = os.path.join(TESTDATADIR, 'SAM/wind_gen_standard_losses_0.json')

pp = ProjectPoints.lat_lon_coords(lat_lons, res_file, sam_file)
gen = Gen.reV_run('windpower', pp, sam_file, res_file,
                  max_workers=1, out_fpath=None,
                  output_request=('cf_mean', 'cf_profile'))
display(gen.out['cf_profile'])

/Users/lijjumathew/opt/anaconda3/lib/python3.8/site-packages/reV/SAM/version_checker.py:101: PySAMVersionWarning: It appears old SAM v1 keys are being used. Updated key "wind_farm_losses_percent" to "turb_generic_loss".
  warn(m, PySAMVersionWarning)


array([[0.06 , 0.213, 0.265, 0.312, 0.83 , 0.827],
       [0.14 , 0.364, 0.438, 0.613, 0.831, 0.833],
       [0.023, 0.685, 0.371, 0.562, 0.833, 0.833],
       ...,
       [0.792, 0.589, 0.833, 0.833, 0.833, 0.833],
       [0.833, 0.833, 0.832, 0.798, 0.833, 0.833],
       [0.711, 0.827, 0.655, 0.19 , 0.833, 0.833]], dtype=float32)

#### pvwatts

NOTE: ``pvwattsv5`` and ``pvwattsv7`` are both available from reV.

Compute pv mean capacity factors and mean levelized cost of energy for all resource gids in New York county New York:

In [7]:
import os
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

regions = {'Arlington': 'county'}

res_file = '/nrel/nsrdb/v3/nsrdb_2012.h5'  # HSDS 'file' path
sam_file = os.path.join(TESTDATADIR, 'SAM/naris_pv_1axis_inv13.json')

pp = ProjectPoints.regions(regions, res_file, sam_file)
gen = Gen.reV_run('pvwattsv5', pp, sam_file, res_file,
                  max_workers=1, out_fpath=None,
                  output_request=('cf_mean', 'lcoe_fcr'))
display(gen.out['cf_mean'])
display(gen.out['lcoe_fcr'])

array([0.18077604, 0.17945854, 0.18129802, 0.17552263], dtype=float32)

array([128.7464 , 129.6916 , 128.37573, 132.59981], dtype=float32)

In [9]:
display(gen)

In [4]:
from rex import init_logger
import h5pyd
import logging

if __name__ == '__main__':
    logger = logging.getLogger(__name__)
    init_logger(__name__, log_level='DEBUG')
    fp = '/nrel/nsrdb/v3/nsrdb_2019.h5'
    with h5pyd.File(fp, logger=__name__) as f:
        data = f['ghi'][:, 0]
    print(data.shape)

DEBUG - 2022-05-07 21:02:05,894 [httpconn.py:186] : HttpCon.init(domain: /nrel/nsrdb/v3/nsrdb_2019.h5 use_session: True use_cache: True retries: 10)


DEBUG:__main__:HttpCon.init(domain: /nrel/nsrdb/v3/nsrdb_2019.h5 use_session: True use_cache: True retries: 10)


DEBUG - 2022-05-07 21:02:05,896 [httpconn.py:309] : using api key


DEBUG:__main__:using api key


DEBUG - 2022-05-07 21:02:05,897 [httpconn.py:376] : GET: / [/nrel/nsrdb/v3/nsrdb_2019.h5] bucket: None


...

INFO - 2022-05-07 21:02:06,698 [dataset.py:1119] : got arr: (17520,), cleaning up shape!


INFO:__main__:got arr: (17520,), cleaning up shape!


DEBUG - 2022-05-07 21:02:06,700 [files.py:555] : close, mode: r


DEBUG:__main__:close, mode: r


(17520,)


In [28]:
f = h5pyd.File(fp, logger=__name__)
import json
print(json.load(f))

DEBUG - 2022-01-24 20:53:36,310 [httpconn.py:186] : HttpCon.init(domain: /nrel/nsrdb/v3/nsrdb_2019.h5 use_session: True use_cache: True retries: 10)


DEBUG:__main__:HttpCon.init(domain: /nrel/nsrdb/v3/nsrdb_2019.h5 use_session: True use_cache: True retries: 10)


DEBUG - 2022-01-24 20:53:36,311 [httpconn.py:309] : using api key


DEBUG:__main__:using api key


DEBUG - 2022-01-24 20:53:36,313 [httpconn.py:376] : GET: / [/nrel/nsrdb/v3/nsrdb_2019.h5] bucket: None


...

DEBUG - 2022-01-24 20:53:37,034 [httpconn.py:425] : adding / to cache


DEBUG:__main__:adding / to cache


INFO - 2022-01-24 20:53:37,040 [httpconn.py:430] : got domain json: {'root': 'g-8833513a-3ffcfdba-00bb-d9b2b7-747532', 'class': 'domain', 'owner': 'nrel_admin', 'created': 1595000129.3137813, 'limits': {'min_chunk_size': 1048576, 'max_chunk_size': 4194304, 'max_request_size': 104857600, 'max_chunks_per_request': 1000}, 'compressors': ['blosclz', 'lz4', 'lz4hc', 'snappy', 'gzip', 'zstd'], 'version': '0.7.0beta', 'lastModified': 1595001544.389294, 'domain_objs': {'g-8833513a-3ffcfdba-00bb-d9b2b7-747532': {'id': 'g-8833513a-3ffcfdba-00bb-d9b2b7-747532', 'root': 'g-8833513a-3ffcfdba-00bb-d9b2b7-747532', 'created': 1595000129.2215984, 'lastModified': 1595001522.2152836, 'linkCount': 28, 'links': {'air_temperature': {'class': 'H5L_TYPE_HARD', 'id': 'd-8833513a-3ffcfdba-9a9e-be7c34-b85178', 'created': 1595000178.0457559}, 'alpha': {'class': 'H5L_TYPE_HARD', 'id': 'd-8833513a-3ffcfdba-a457-3a278a-8dda44', 'created': 1595000197.1176045}, 'aod': {'class': 'H5L_TYPE_HARD', 'id': 'd-8833513a-3ffcf

INFO:__main__:got domain json: {'root': 'g-8833513a-3ffcfdba-00bb-d9b2b7-747532', 'class': 'domain', 'owner': 'nrel_admin', 'created': 1595000129.3137813, 'limits': {'min_chunk_size': 1048576, 'max_chunk_size': 4194304, 'max_request_size': 104857600, 'max_chunks_per_request': 1000}, 'compressors': ['blosclz', 'lz4', 'lz4hc', 'snappy', 'gzip', 'zstd'], 'version': '0.7.0beta', 'lastModified': 1595001544.389294, 'domain_objs': {'g-8833513a-3ffcfdba-00bb-d9b2b7-747532': {'id': 'g-8833513a-3ffcfdba-00bb-d9b2b7-747532', 'root': 'g-8833513a-3ffcfdba-00bb-d9b2b7-747532', 'created': 1595000129.2215984, 'lastModified': 1595001522.2152836, 'linkCount': 28, 'links': {'air_temperature': {'class': 'H5L_TYPE_HARD', 'id': 'd-8833513a-3ffcfdba-9a9e-be7c34-b85178', 'created': 1595000178.0457559}, 'alpha': {'class': 'H5L_TYPE_HARD', 'id': 'd-8833513a-3ffcfdba-a457-3a278a-8dda44', 'created': 1595000197.1176045}, 'aod': {'class': 'H5L_TYPE_HARD', 'id': 'd-8833513a-3ffcfdba-3c5d-31cf78-214988', 'created': 1

AttributeError: 'File' object has no attribute 'read'

In [3]:
[key for key in f.keys()]

AttributeError: 'NoneType' object has no attribute 'getObjDb'

In [35]:
dataset = h5pyd.File(fp, 'r')

In [36]:
dataset.keys()

KeysView(<HDF5 file "nsrdb_2019.h5" (mode r)>)

In [42]:
from rex import NSRDBX

nsrdb_file = '/nrel/nsrdb/v3/nsrdb_2012.h5'
nrel = (39.741931, -105.169891)
with NSRDBX(nsrdb_file, hsds=True) as f:
    nrel_dni = f.get_lat_lon_df('dni', nrel)

In [46]:
# Extract time-series data for a single site
import h5pyd
import pandas as pd

# Open .h5 file
with h5pyd.File('/nrel/nsrdb/v3/nsrdb_2012.h5', mode='r') as f:
    # Extract time_index and convert to datetime
    # NOTE: time_index is saved as byte-strings and must be decoded
    time_index = pd.to_datetime(f['time_index'][...].astype(str))
    # Initialize DataFrame to store time-series data
    time_series = pd.DataFrame(index=time_index)
    # Extract variables needed to compute generation from SAM:
    for var in ['dni', 'dhi', 'air_temperature', 'wind_speed']:
    	# Get dataset
    	ds = f[var]
    	# Extract scale factor
    	scale_factor = ds.attrs['psm_scale_factor']
    	# Extract site 100 and add to DataFrame
    	time_series[var] = ds[:, 100] / scale_factor


In [47]:
time_series

,dni,dhi,air_temperature,wind_speed
2012-01-01 00:00:00,293.0,485.0,26.0,3.4
2012-01-01 00:30:00,347.0,406.0,26.0,3.0
2012-01-01 01:00:00,339.0,376.0,26.0,2.8
2012-01-01 01:30:00,330.0,340.0,26.0,2.5
2012-01-01 02:00:00,519.0,234.0,26.0,2.3
...,...,...,...,...
2012-12-31 21:30:00,431.0,407.0,26.0,2.3
2012-12-31 22:00:00,443.0,410.0,26.0,2.2
2012-12-31 22:30:00,446.0,411.0,26.0,2.0
2012-12-31 23:00:00,446.0,409.0,26.0,1.9


In [ ]:
Ja